<a href="https://colab.research.google.com/github/sayed0405035/CV_pdf_data_extraction/blob/main/data_from_sections_2_9_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import pickle
import os

import re

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
import seaborn as sns

#from pdfminer.high_level import extract_text
import subprocess
nltk.download('stopwords')

import json

from IPython.utils.text import string


import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

import pickle
import os






[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [143]:

#pdf_path = '/content/drive/MyDrive/private dataset(no sharing)/agaspar_cv.pdf'
#pdf_path = '/content/drive/MyDrive/private dataset(no sharing)/Alison-M-Meadow-cv.pdf'


#pdf_path = '/content/drive/MyDrive/private dataset(no sharing)/CV_brief.pdf'

#pdf_path = '/content/drive/MyDrive/private dataset(no sharing)/Canales_Robert_CV.pdf'


pdf_path = '/content/drive/MyDrive/private dataset(no sharing)/Hoit CV (4-11-16).pdf'






file_name = os.path.basename(pdf_path)
file_name_without_extension = os.path.splitext(file_name)[0]
print(file_name_without_extension)


pickle_path = '/content/drive/MyDrive/CV_extract_data/pickle_data/' + file_name_without_extension +"_section" + '.pickle'








Hoit CV (4-11-16)


In [144]:
with open(pickle_path, 'rb') as file:
    loaded_dict = pickle.load(file)

print(loaded_dict)


{'contact information': ['Hoit - 1 JEANNETTE DEE HOIT Department of Speech, Language, and Hearing Sciences 5631 N. Campbell Ave. P.O. Box 210071 Tucson, Arizona 85718 University of Arizona phone: (520) 615-1913 Tucson, Arizona 85721 phone: (520) 621-7064 E-mail: hoit@email.arizona.edu fax: (520) 626-1364'], 'education': ['EDUCATION1989-90 Postdoctoral Study; Physiology Program, Harvard University, and Speech', 'CERTIFICATION2002-present State of Arizona Licensure in Speech-Language Pathology (#SLP0264) 1981 State of California Licensure in Speech-Language Pathology 1981 Certificate of Clinical Competence in Speech-Language Pathology from the American Speech-Language-Hearing Association 1979 State of California Licensure in School Audiometry', 'Education materials2005-07 Editor, American Journal of Speech-Language Pathology 2002-present Ad hoc Reviewer, Journal of Ear and Hearing, American Journal of Respiratory and Hoit - 8 Critical Care Medicine, Journal of Applied Physiology, Chest, 

In [145]:

def arrange_load_dict(loaded_dict):
    new_dict_for_division = dict()

    for key, value in loaded_dict.items():
    #print("key ::",key,"  value ::", value)
        current_compound_line = ""
        check_compound_line = False
        new_dict_for_division[key] = []


        for line in value:
            line = line.strip()
        # print(type(line),line)
            if len(line.split(' '))==1:
            #current_compound_line += " " + line
                check_compound_line = True
            else:
                if check_compound_line:
            #print("check_compound_line ::",current_compound_line)
                    current_compound_line += " " + line
                    check_compound_line = False
                else:
 #                print("line ::",line)
                    new_dict_for_division[key].append(line)

        #     print(" line :::  ",line)
    #print("check_compound_line ::",current_compound_line)
        current_compound_line = current_compound_line.strip()
        if len(current_compound_line)>0:
            new_dict_for_division[key].append(current_compound_line)
        #break
    return new_dict_for_division





In [146]:
new_dict_for_division = arrange_load_dict(loaded_dict)

In [147]:
new_dict_for_division['contact information']

['Hoit - 1 JEANNETTE DEE HOIT Department of Speech, Language, and Hearing Sciences 5631 N. Campbell Ave. P.O. Box 210071 Tucson, Arizona 85718 University of Arizona phone: (520) 615-1913 Tucson, Arizona 85721 phone: (520) 621-7064 E-mail: hoit@email.arizona.edu fax: (520) 626-1364']

In [148]:
################################## Done processing for sections

### serial no. ## Section name ----### parts of the section ------------ Status

# 1 ---->>> contact information -->> names, e-mail, phone number         -->> ok
# 2 ---->>> education           -->> year, degree, organization, advisor -->> ok
# 3 ---->>> research areas      -->>            need more work, not good enough
# 4 ---->>> all_appoinments     -->> need to check with data
# 5 ---->>> proposals/grants    -->> not ok for some data
# 6 ---->>> teaching/ outreach  -->> need to check with data
# 7 ---->>> students            -->> not very good, take a look
# 8 ---->>> honors/awards       -->> good
# 9 ---->>> selected talks/meetings  -->> need testing with data
# 10 --->>> services            -->> good
# 11 --->>> observing experience   -->> need testing with data
# 12 --->>> publications           -->> need testing with data
# 13 --->>> co-authored papers     -->> not implemented yet
# 14 --->>> conference abstracts and proceedings   -->> not good result, check needed












In [149]:
new_dict_for_division.keys()

dict_keys(['contact information', 'education', 'research areas', 'services', 'honors/awards', 'teaching/ outreach', 'proposals/grants', 'publications', 'students'])

In [62]:


def extract_names(txt):
    person_names = []

    for sent in nltk.sent_tokenize(txt):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label') and chunk.label() == 'PERSON':
                person_names.append(
                    ' '.join(chunk_leave[0] for chunk_leave in chunk.leaves())
                )
    if len(person_names)==0:

        nlp = spacy.load("en_core_web_sm")
        doc = nlp(txt)

        for entity in doc.ents:
            if entity.label_ == "PERSON":
                person_names.append(entity.text)

    #return None


    return person_names


In [63]:
#E-MAIL


def get_email_addresses(string):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return r.findall(string)



In [64]:
#PHONE NUMBER


def get_phone_numbers(string):
    r = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
    phone_numbers = r.findall(string)
    return [re.sub(r'\D', '', num) for num in phone_numbers]


In [65]:

def handled_contact_information(sent_contact_information_list):

    all_contact_data = dict()
    all_contact_data["Names"] = []
    all_contact_data["email"] = []
    all_contact_data["phone_number"] = []
    for total_info in sent_contact_information_list:


        names = extract_names(total_info)
        email = get_email_addresses(total_info)
        phone_number = get_phone_numbers(total_info)
        if len(names)!=0:
            all_contact_data["Names"].extend(names)
        if len(email)!=0:
            all_contact_data["email"].extend(email)
        if len(email)!=0:
            all_contact_data["phone_number"].extend(phone_number)

        return all_contact_data




In [ ]:


contact_informations = handled_contact_information(new_dict_for_division['contact information'])


contact_informations



In [151]:


def handle_education(sent_text):

    sent_text = re.sub(r"^(?i)education", "", sent_text)

    #sent_text = re.sub(r"(?i)education", " ", sent_text)
    #print(sent_text)

    # Regular expression patterns for extracting information
    pattern = r"(\d{4})\s*(Ph\.D\.|M\.Sc\.|B\.Sc\.)\s*,\s*([^,]+),\s*([^,]+)"

    # Extract information using regular expressions
    matches = re.findall(pattern, sent_text)
    #print(matches)
    # Process the extracted matches
    education_entries = []
    for match in matches:
        entry = {
            "degree": match[1],
            "year": match[0],
            "field_of_study": match[2].strip(),
            "university": match[3].strip(),
            "thesis": None,
            "advisor": None
        }
        education_entries.append(entry)

    # Find the thesis and advisor for each entry
    for i, entry in enumerate(education_entries):
        thesis_pattern = re.escape(entry["degree"]) + r".*?Thesis:\s*“([^”]+)”"
        thesis_match = re.search(thesis_pattern, sent_text)
        #print(thesis_match)
        if thesis_match:
            education_entries[i]["thesis"] = thesis_match.group(1)

        advisor_pattern = re.escape(entry["degree"]) + r".*?Advisor:\s*([^,]+)"
        advisor_match = re.search(advisor_pattern, sent_text)
        if advisor_match:
            education_entries[i]["advisor"] = advisor_match.group(1)
        #print(advisor_match)
    # Print the extracted information
    # for entry in education_entries:
    #     print("Degree:", entry["degree"])
    #     print("Year:", entry["year"])
    #     print("Field of Study:", entry["field_of_study"])
    #     print("University:", entry["university"])
    #     print("Thesis:", entry["thesis"])
    #     print("Advisor:", entry["advisor"])
    #     print()

    if len(education_entries)==0:
        #print("Zero")
        entries = re.split(r'(\b\d{4}\b)', sent_text)
        entries = [entry.strip() for entry in entries if entry.strip()]
        #print("entries ::",entries)
        year = []
        degree = []
        organization = []
        advisor = []
        for i in range(0, len(entries), 2):
            if i==len(entries)-1:
                continue
            if entries[i+1].isdigit():
                year = entries[i+1]
                education_info = entries[i]
            else:
                year = entries[i]
                education_info = entries[i+1]

            #print("education_info ::",education_info)

            degree_match = re.search(r'([A-Za-z.]+\s*\.)', education_info)
            if degree_match:
                degree = degree_match.group(1)

                organization_match = re.search(r'{}(.+?)(?=\b\d{{4}}\b|$)'.format(degree), education_info)
                if organization_match:
                    organization = organization_match.group(1).strip()

                    university_of_match = re.search(r'University of (\w+)', organization, re.IGNORECASE)
                    university_match = re.search(r"(\w+) University", organization, re.IGNORECASE)

                    if university_of_match:
                        organization = "University of {}".format(university_of_match.group(1))
                    elif university_match:
                        organization = "{} University".format(university_match.group(1))

            advisor_match = re.search(r'Advisor:\s*(.*)', education_info)
            advisor = advisor_match.group(1).strip() if advisor_match else None

            education = {
                "year": year,
                "degree": degree,
                "organization": organization,
                "advisor": advisor
                }
            education_entries.append(education)

    if len(education_entries)>0:

        return education_entries

    sent_text = re.sub(r"^(?i)education", "", sent_text)

    #sent_text = re.sub(r"(?i)education", " ", sent_text)
    #print(sent_text)

    # Regular expression patterns for extracting information
    pattern = r"(\d{4})\s*(Ph\.D\.|M\.Sc\.|B\.Sc\.)\s*,\s*([^,]+),\s*([^,]+)"

    # Extract information using regular expressions
    matches = re.findall(pattern, sent_text)
    #print(matches)
    # Process the extracted matches
    education_entries = []
    for match in matches:
        entry = {
            "degree": match[1],
            "year": match[0],
            "field_of_study": match[2].strip(),
            "university": match[3].strip(),
            "thesis": None,
            "advisor": None
        }
        education_entries.append(entry)

    # Find the thesis and advisor for each entry
    for i, entry in enumerate(education_entries):
        thesis_pattern = re.escape(entry["degree"]) + r".*?Thesis:\s*“([^”]+)”"
        thesis_match = re.search(thesis_pattern, sent_text)
        #print(thesis_match)
        if thesis_match:
            education_entries[i]["thesis"] = thesis_match.group(1)

        advisor_pattern = re.escape(entry["degree"]) + r".*?Advisor:\s*([^,]+)"
        advisor_match = re.search(advisor_pattern, sent_text)
        if advisor_match:
            education_entries[i]["advisor"] = advisor_match.group(1)
        #print(advisor_match)
    # Print the extracted information
    # for entry in education_entries:
    #     print("Degree:", entry["degree"])
    #     print("Year:", entry["year"])
    #     print("Field of Study:", entry["field_of_study"])
    #     print("University:", entry["university"])
    #     print("Thesis:", entry["thesis"])
    #     print("Advisor:", entry["advisor"])
    #     print()

    if len(education_entries)==0:
        #print("Zero")
        entries = re.split(r'(\b\d{4}\b)', sent_text)
        entries = [entry.strip() for entry in entries if entry.strip()]
        #print("entries ::",entries)

        for i in range(0, len(entries), 2):
            if i==len(entries)-1:
                continue
            if entries[i+1].isdigit():
                year = entries[i+1]
                education_info = entries[i]
            else:
                year = entries[i]
                education_info = entries[i+1]

            #print("education_info ::",education_info)

            degree_match = re.search(r'([A-Za-z.]+\s*\.)', education_info)
            if degree_match:
                degree = degree_match.group(1)

            organization_match = re.search(r'{}(.+?)(?=\b\d{{4}}\b|$)'.format(degree), education_info)
            if organization_match:
                organization = organization_match.group(1).strip()

                university_of_match = re.search(r'University of (\w+)', organization, re.IGNORECASE)
                university_match = re.search(r"(\w+) University", organization, re.IGNORECASE)

                if university_of_match:
                    organization = "University of {}".format(university_of_match.group(1))
                elif university_match:
                    organization = "{} University".format(university_match.group(1))

            advisor_match = re.search(r'Advisor:\s*(.*)', education_info)
            advisor = advisor_match.group(1).strip() if advisor_match else None

            education = {
                "year": year,
                "degree": degree,
                "organization": organization,
                "advisor": advisor
                }
            education_entries.append(education)


    return education_entries






In [68]:


def handled_all_educations(sent_educations_list):
    all_educations = []
    for item in sent_educations_list:
        temp_educations_list = handle_education(item)
        if len(temp_educations_list)>0:
            all_educations.append(temp_educations_list)
    return all_educations





In [ ]:
all_educations_returned_list = handled_all_educations(new_dict_for_division['education'])



all_educations_returned_list



In [69]:


def handle_current_research_projects(sent_text):

     # Dictionary to store research data
    current_research_data = []

    sent_text = re.sub(r"^(?i)current", "", sent_text)
    sent_text = re.sub(r"^(?i)research", "", sent_text)
    sent_text = re.sub(r"^(?i)projects", "", sent_text)

    if "\x0f" not in sent_text:
        return current_research_data

    current_research_project_data_list_temp = sent_text.split("\x0f")


    current_research_project_data_list = []

    for item in current_research_project_data_list_temp:
        if item.strip() != "":
            current_research_project_data_list.append(item)
        # print(current_research_project_data_list)




    # Extract research topic and duration from each item
    for item in current_research_project_data_list:
        doc = nlp(item)

        # Extract research topic
        research_topic = doc.text.strip()

        # Extract duration using regular expressions
        duration_pattern = r"(\d+(?:\.\d+)?)\s*hours"
        duration_match = re.search(duration_pattern, item, re.IGNORECASE)

        duration = ""
        if duration_match:
            duration = duration_match.group(1)

        # Create a dictionary to store the topic and duration
        result = {
            "topic": research_topic,
            "duration": duration
        }

        # Append the result to the research data list
        current_research_data.append(result)

        # print("Research Topic:", research_topic)
        # print("Duration:", duration)
        # print("--------------------")
    return current_research_data




In [70]:


def handle_research_areas(sent_text):


    sent_text = re.sub(r"^(?i)research areas", "", sent_text)

    #sent_text = re.sub(r"(?i)research areas", " ", sent_text)
    #    print(sent_text)
    list_of_research_areas = sent_text.split(",")
    return list_of_research_areas





In [72]:



def handle_all_research_area(given_list):
    returned_list = []
    for item in given_list:
        res = handle_current_research_projects(item)
        if len(res)>0:
            returned_list.append(res)
    if len(returned_list)==0:
        for item in given_list:
            res = handle_research_areas(item)
            if len(res)>0:
                returned_list.append(res)

    return returned_list





In [ ]:
all_handled_research = handle_all_research_area(new_dict_for_division['research areas'])


all_handled_research


In [73]:


def handle_appoinments(sent_text):

    sent_text = re.sub(r"^(?i)appointments", "", sent_text)
    #sent_text = re.sub(r"(?i)appointments", " ", sent_text)
    # print(sent_text)

    appointments = []

        # Extract information using regex
    matches = re.findall(r"(\d{4})\s?-\s?(current|\d{4})\s?([^,]+),\s([^0-9]+)", sent_text)
    for match in matches:
        start_date = int(match[0])
        if match[1] == "current":
            end_date = "current"
        else:
            end_date = int(match[1])
        position = match[2].strip()
        organization = match[3].strip()

        appointments.append({
            "start_date": start_date,
            "end_date": end_date,
            "position": position,
            "organization": organization
        })

        # Print the extracted information
    # for appointment in appointments:
    #     print("Start Date:", appointment["start_date"])
    #     print("End Date:", appointment["end_date"])
    #     print("Position:", appointment["position"])
    #     print("Organization:", appointment["organization"])
    #     print()
    return appointments




In [74]:


def handle_all_appoinments(sent_list):
    result = []
    for item in sent_list:
        res = handle_appoinments(item)
        if len(res)>0:
            result.append(res)
    return result




In [ ]:
all_appoinments = handle_all_appoinments(new_dict_for_division['appointments'])

all_appoinments



In [75]:

def handled_grants(sent_text):

    sent_text = re.sub(r"^(?i)grants", "", sent_text)
    # print(sent_text)
  # Text containing the grant data
  #grant_data = "(Co-I)2016 Debris Disk Variability - Exploring the Diverse Outcomes of Large Collisions during the Eras of Oligarchic and Chaotic Growth NASA ADAP ($200k) 2016 Probing Terrestrial Planet Formation with Extreme Disk Variability Total award: 120 h with Spitzer Space Telescope 2015 Contrasting Exoplanetary Systems by Resolving the Debris Disk around Denebola Total award: 8.5 h with ALMA 2014 Earths in Other Solar Systems: Toward forming and discovering planets with biocritical ingredients NASA Astrobiology Institute proposal ($7 million) 2014 Decoding Debris System Substructures: Imprints of Planets/Planetesimals and Signatures of Extrinsic Inﬂuences on Material in Ring-Like Disks Total award: 40 orbits on HST/STIS ($280k) 2014 Pushing to 8 AU in the archetypal protoplanetary disk of TW Hya Total award: 8 orbits on HST/STIS 2010 Diversity in Debris Disks: Snapshots of Planetary System Evolution Total award: 109 h with Spitzer Space Telescope ($266k) 2009 The Master Sample of Spitzer Debris Disk Measurements Total award: 103 h with Spitzer Space Telescope ($330k)"

   # Extract the grants and their corresponding awards using regular expressions
    pattern = r'(?P<year>\d{4})\s(?P<grant>.*?)\sTotal award:\s(?P<award>.*?)\s'
    matches = re.findall(pattern, sent_text)

    # Initialize a dictionary to store grants by year
    grants_by_year = {}

   # Organize the grants and awards by year
    for match in matches:
        year = int(match[0])
        grant = match[1]
        award = match[2]
        if year in grants_by_year:
            grants_by_year[year].append({"grant": grant, "award": award})
        else:
            grants_by_year[year] = [{"grant": grant, "award": award}]

  # Print the grants and awards organized by year
    # for year, grants in grants_by_year.items():
    #     print(f"{year}:")
    #     for grant in grants:
    #         print("- Grant:", grant["grant"])
    #         print("  Award:", grant["award"])
    #         print()
    return grants_by_year



In [76]:
# it gives more correct data

def handled_grants_02(sent_text):

 # Text containing the grant data
 #grant_data = "Grants (PI)2012 Pointing the Finger: Calibrating the Hidden Features of STIS and Enabling New Coronagraphy at Separations of 0.1500 Total award: 6 orbits on HST/STIS ($88k) 2011 Observing Young Debris Disks with SPIRE Total award: 2.2 hours of OT2 time with Herschel (Not carried out due to EOM) Grants (Co-I)2016 Debris Disk Variability - Exploring the Diverse Outcomes of Large Collisions during the Eras of Oligarchic and Chaotic Growth NASA ADAP ($200k) 2016 Probing Terrestrial Planet Formation with Extreme Disk Variability Total award: 120 h with Spitzer Space Telescope"

    sent_text = re.sub(r"^(?i)grants", "", sent_text)
    # print(sent_text)
 # Find all the years in the grant data
    years = re.findall(r"\b(?!19\d{2})\d{4}\b", sent_text)

 # Divide the grant data by years
    grant_texts = re.split(r"\b(?!19\d{2})\d{4}\b", sent_text)[1:]

 # Initialize a dictionary to store grants by year
    grants_by_year = {}

 # Assign grants to their respective years
    for i, year in enumerate(years):
        grant_text = grant_texts[i].strip()
        if year in grants_by_year:
            grants_by_year[year].append(grant_text)
        else:
            grants_by_year[year] = [grant_text]

 # Print the grants organized by year
    # for year, grants in grants_by_year.items():
    #     print(f"{year}:")
    #     for grant in grants:
    #         print("- Grant:", grant)
    #     print()
    return grants_by_year



In [77]:


def handle_all_grants(sent_list):
    result = []
    for item in sent_list:
        res = handled_grants(item)
        if len(res)>0:
            result.append(res)
    return result




In [78]:


def handle_all_grants_02(sent_list):
    result = []
    for item in sent_list:
        res = handled_grants_02(item)
        if len(res)>0:
            result.append(res)
    return result




In [ ]:
all_awards_grants = handle_all_grants(new_dict_for_division['proposals/grants'])


all_awards_grants




In [ ]:
all_awards_grants_02 = handle_all_grants_02(new_dict_for_division['proposals/grants'])


all_awards_grants_02




In [79]:

def handled_teaching_outreach(sent_text):


    sent_text = re.sub(r"^(?i)teaching", "", sent_text)
    sent_text = re.sub(r"^(?i)outreach", "", sent_text)




 #data = "2017 AST101IN: Solar System (hybrid course), Pima Community College 2010 TA for ASTR 202: Life in the Universe, UofA http://merope.as.arizona.edu/˜agaspar/ASTR202 Professor: Dr. Christopher Walker 2009 TA for ASTR 250: Fundamentals of Astronomy, UofA Professor: Dr. Yancy Shirley 2006 Astronomical Lab II, University of Szeged 2005 Astronomical Lab I, University of Szeged 2001 - 2006 Open nights at the Public Observatory, Szeged, Hungary"
    data = sent_text.strip()
 # Use regular expressions to split the string by year or year range
    year_pattern = r"\b(\d{4})(?:\s?-\s?(\d{4}))?\b"
    data_split_temp = re.split(year_pattern, data)

    data_split = []
    for item in data_split_temp:
        if item!="":
            data_split.append(item)

    # print("data_split ::",data_split)

 # Filter out any empty strings resulting from the split
    data_split = list(filter(None, data_split))

 #print(" len(data_split) ::",len(data_split))

 # Create a dictionary to store the text for each year
    year_text_dict = {}

    year = ""
    text = ""
    check = True
 # Iterate over the data and populate the dictionary
    count = 2

    for item in data_split :
    #if i % 2 == 0:
        if item==None or item.isdigit():
            if check:
                year = ""
                check = False
            if item!=None:
                if year!="":
                    year += "-" + item
                else:
                    year += item
            if text!="":
                if year not in year_text_dict:
                    year_text_dict[year] = text.strip()
                else:
                    t= year + " no. " + str(count)
                    count +=1
                    year_text_dict[t] = text.strip()
                #year_text_dict[year] = text.strip()
                text = ""

        else:
            text += " " + item
            check = True


        # if i == len(data_split) - 2:
        #     year_text_dict[year] = text.strip()
        # else:
        #     next_year = data_split[i + 2]
    if year not in year_text_dict:
        year_text_dict[year] = text.strip()
    else:
        t= year + " no. " + str(count)
        count +=1
        year_text_dict[t] = text.strip()


 # Print the dictionary with year-text pairs
    # for year, text in year_text_dict.items():
    #     print(f"{year}: {text}")
    return year_text_dict



In [80]:


def all_handled_teaching_outreach(sent_list):
    result = []
    for item in sent_list:
        res = handled_teaching_outreach(item)
        if len(res)>0:
            result.append(res)
    return result





In [ ]:
handled_teaching_outreach_list = all_handled_teaching_outreach(new_dict_for_division['teaching/ outreach'])


handled_teaching_outreach_list



In [81]:


def extract_student_data(students_list):
    student_data = []
    degree_pattern = r'(M\.Sc\.|Ph\.D\.)'
    for item_temp in students_list:


        item_temp = re.sub(r"^(?i)students", "", item_temp)
        item_temp_list = item_temp.split("\x0f")
        if "\x0f" not in item_temp:
            item_temp_list = item_temp.split("•")
        if len(item_temp_list)==0:
            item_temp_list = item_temp.split(".")

        item_list = []
        for it in item_temp_list:
            if it.strip()!="":
                item_list.append(it)
        # print(item_list)
        for item in item_list:
            doc = nlp(item)

            # Extract degree using modified regular expression pattern
            degree_match = re.search(degree_pattern, item)
            # print("item :::",item)
            # print("degree_match  ::",degree_match)

            degree = ""
            if degree_match:
                degree = degree_match.group(0)

            # Extract student name and organization using NER (Named Entity Recognition)
            student_name = ""
            organization_name = ""

            for ent in doc.ents:
                if ent.label_ == "PERSON":
                    student_name = ent.text
                elif ent.label_ == "ORG":
                    organization_name = ent.text

            # Create a dictionary to store the extracted data
            result = {
                "degree": degree,
                "student_name": student_name,
                "organization_name": organization_name
            }

            # Append the result to the student data list
            student_data.append(result)

    return student_data




In [ ]:
student_data_list = extract_student_data(new_dict_for_division['students'])

student_data_list


In [82]:

def handled_honors_awards(sent_text_list):
    honors_awards_all_sum = []
    for sent_text in sent_text_list:
        sent_text = re.sub(r"^(?i)awards", "", sent_text)
        sent_text = re.sub(r"^(?i)honors", "", sent_text)

        honors = re.findall(r"(\d{4})\s(.*?)\s\d{4}", sent_text)

        honors_awards_all = []

        for year, honor in honors:
            honors_awards = {
                "Year:": year,
                "Honors/Awards:": honor
            }
            # print("Year:", year)
            # print("Honors:", honor)
            # print()
            honors_awards_all.append(honors_awards)
        honors_awards_all_sum.append(honors_awards_all)


    return honors_awards_all_sum




In [ ]:
all_honors_rewards = handled_honors_awards(new_dict_for_division['honors/awards'])


all_honors_rewards



In [83]:


def handled_talks_meetings(sent_text_list):
    talks_meetings_all_sum = []
    for sent_text in sent_text_list:
        sent_text = re.sub(r"^(?i)meetings", "", sent_text)
        sent_text = re.sub(r"^(?i)talks", "", sent_text)
        sent_text = re.sub(r"^(?i)selected", "", sent_text)

        talks_meetings = re.findall(r"(\d{4})\s(.*?)\s\d{4}", sent_text)

        talks_meetings_all = []

        for year, talks_meeting in talks_meetings:
            talks_meetings_temp = {
                "Year:": year,
                "Talks/Meetings:": talks_meeting
            }
            # print("Year:", year)
            # print("Talks/Meetings:", talks_meeting)
            # print()
            talks_meetings_all.append(talks_meetings_temp)
        talks_meetings_all_sum.append(talks_meetings_all)


    return talks_meetings_all_sum





In [ ]:
talks_meetings = handled_talks_meetings(new_dict_for_division['selected talks/meetings'])


talks_meetings




In [84]:

def handled_services(sent_text_list):
    services_all_sum = []
    count = 1
    for sent_text in sent_text_list:
        sent_text = re.sub(r"^(?i)services", "", sent_text.strip())
        sent_text = sent_text.strip()
        if "\x0f" in sent_text:
            services_sum = []
            services_list_temp = sent_text.split("\x0f")
            for service in services_list_temp:
                if len(service.strip())>2:
                    service_dict = {
                        "Service number" : count,
                        "Service description" : service
                    }
                    count +=1
                    services_sum.append(service_dict)
            if len(services_sum)>0:
                services_all_sum.append(services_sum)
    return services_all_sum




In [85]:

def extract_information_for_service(text):
    # Regular expression pattern for matching years
    year_pattern = r"\b(?:\d{4}\s?(?:–|-)\s?(?:\d{4}|present)|\d{4})\b"

    # Regular expression pattern for matching organization names like "University of Texas"
    org_pattern = re.compile(r"University of [A-Za-z]+", re.IGNORECASE)

    # Load the English language model in spaCy
    nlp = spacy.load("en_core_web_sm")

    years = re.findall(year_pattern, text)
    segments = re.split(year_pattern, text)

    extracted_information = []

    for i in range(len(years)):
        segment_text = segments[i].strip()
        doc = nlp(segment_text)

        # Extract organization names
        organizations = []
        for entity in doc.ents:
            if entity.label_ == "ORG" or org_pattern.match(entity.text):
                organizations.append(entity.text)

        # Extract the remaining text
        remaining_text = " ".join([token.text for token in doc if not token.ent_type_])

        # Create a dictionary with year, organization, and remaining text
        segment_info = {
            "year": years[i],
            "organization": organizations,
            "remaining_text": remaining_text
        }

        extracted_information.append(segment_info)

    # Add the remaining segment after the last year
    remaining_segment = segments[-1].strip()
    extracted_information.append({
        "year": None,
        "organization": [],
        "remaining_text": remaining_segment
    })

    return extracted_information




In [86]:

def handled_services_02(sent_text_list):
    services_all_sum = []
    count = 1
    for sent_text in sent_text_list:
        sent_text = re.sub(r"^(?i)services", "", sent_text.strip())
        sent_text = sent_text.strip()
        if "\x0f" in sent_text:
            services_sum = []
            services_list_temp = sent_text.split("\x0f")
            for service in services_list_temp:
                if len(service.strip())>2:
                    service_dict = {
                        "Service number" : count,
                        "Service description" : service
                    }
                    count +=1
                    services_sum.append(service_dict)
            if len(services_sum)>0:
                services_all_sum.append(services_sum)
    if(len(services_all_sum)==0):
        for sent_text in sent_text_list:
            got_data = extract_information_for_service(sent_text)
            if(len(got_data)>0):
                services_all_sum.append(got_data)
    if(len(services_all_sum)==0):
        #for sent_text in sent_text_list:
        got_data = handled_services(sent_text_list)
        if(len(got_data)>0):
            services_all_sum.append(got_data)
    return services_all_sum






In [ ]:
handled_services_list = handled_services_02(new_dict_for_division['services'])


handled_services_list

In [87]:

def handled_observing_experience(sent_text_list):

    all_experience = []

    for observed_experience_list in sent_text_list:

        # Extract durations using regular expression
        duration_pattern = r'\d+\s(?:weeks|nights|orbits)'
        durations = re.findall(duration_pattern, observed_experience_list)


 #        print(durations)


        words = observed_experience_list.split()
        current_experience = ""
    # Assign durations to experiences

        count = 0
        experiences_list = []
        for word in words:
            if durations[count] in current_experience:
                experiences = {}
                experiences["duration"] = durations[count]
                experiences["experience "] = current_experience.replace(durations[count]," ")
                experiences_list.append(experiences)
                current_experience = word + " "
                count +=1
            else:
                current_experience += word + " "
        experiences = {}
        experiences["duration"] = durations[count]
        experiences["experience "] = current_experience.replace(durations[count]," ")
        experiences_list.append(experiences)
        all_experience.append(experiences_list)

 #    print("Experiences:")
    for experiences_temp in all_experience:
        for experiences in experiences_temp:
            print(experiences)

    return all_experience





In [ ]:
observing_experience_list = handled_observing_experience(new_dict_for_division['observing experience'])


observing_experience_list




In [88]:

def handle_single_text_list_of_publications(given_text_list_of_publication):


 #count = 0
    current_publication = ""
    current_serial = 0
    first_check = True
    all_publications = []

    for word in given_text_list_of_publication.split():
    # if count==100:
    #     break
        current_word_check = True
        if "." in word:
            pattern = r"(\d+)\."

            match = re.search(pattern, word)

            if match:
                extracted_integer = int(match.group(1))
                for_first = False
                if extracted_integer<1000:
                    if first_check:
                        first_check = False
                        current_serial = extracted_integer
                        for_first = True
                    if (extracted_integer==current_serial+1) or for_first:
                        if len(current_publication.strip())>3:
                            all_publications.append(current_publication.strip())
                            current_word_check = False
                            current_publication = word
                            current_serial = extracted_integer
                # print(extracted_integer)
                # print("Word no. ::",count,"   ::",word)
        if current_word_check:
            current_publication += " " + word
    #count+=1
    if len(current_publication.strip())>3:
        all_publications.append(current_publication.strip())

 #print(len(all_publications))

    final_all_publications =[]

    for pub in all_publications:

        if "." in pub and len(pub)>20:
            final_all_publications.append(pub)
        #print("publication :::",pub)
    return final_all_publications




In [89]:

def is_token_in_phrases(token, phrases):
    # Check if the token is part of any of the given phrases
    return any(token.text in phrase for phrase in phrases)

def extract_publication_info(text):
    # Process the input text using spaCy
    doc = nlp(text)

    # Initialize variables to store extracted information
    authors = []
    source = []
    topic_tokens = []  # Store tokens for the topic

    for ent in doc.ents:
        if ent.label_ == "PERSON":
            authors.append(ent.text)
        elif ent.label_ == "ORG":
            source.append(ent.text)

    # Find multi-word organizations using dependency parsing
    for token in doc:
        if token.dep_ == "nsubj" and "PERSON" in [child.ent_type_ for child in token.children]:
            authors.append(token.text)
        elif token.dep_ == "appos" and "ORG" in [child.ent_type_ for child in token.head.children]:
            source.append(token.text)

    # Remove authors and source phrases from the text
    for token in doc:
        if not is_token_in_phrases(token, authors) and not is_token_in_phrases(token, source) and len(token.text)>3:
            topic_tokens.append(token.text)

    # Combine topic tokens to create the topic string
    topic = " ".join(topic_tokens)

    # If all the information is missing, return an empty dictionary
    if not authors and not topic.strip() and not source:
        return {}

    # Create a dictionary to store the extracted information
    publication_info = {
        "Authors": authors,
        "Topic": topic.strip(),
        "Organizations": source
    }

    return publication_info



In [90]:


def handle_all_list_of_publications(given_all_list_of_publication):
    all_publications_list_of_dict = []
    for new_list in given_all_list_of_publication:
        gotten_list_from_single_list = handle_single_text_list_of_publications(new_list)
        #print(gotten_list_from_single_list)
        for single_publications_text in gotten_list_from_single_list:
            info_dict = extract_publication_info(single_publications_text)
            all_publications_list_of_dict.append(info_dict)

    return all_publications_list_of_dict




In [91]:


def get_publication_dict(publication_text):

 # Extract the publication title
    title_match = re.search(r"\.\s*(.*?)\s,", publication_text)
    publication_title = title_match.group(1) if title_match else ""

 # Extract the authors
    authors_match = re.search(r",\s*(.*?)\s\d{4},", publication_text)
    authors = [author.strip() for author in authors_match.group(1).split(",")] if authors_match else []

 # Extract the year
    year_match = re.search(r"(\d{4}),", publication_text)
    year = year_match.group(1) if year_match else ""

 # Extract the journal name
    journal_match = re.search(r"\d{4},\s(.*?),\d{3},", publication_text)
    journal = journal_match.group(1) if journal_match else ""

 # Extract the volume
    volume_match = re.search(r",(\d{3}),", publication_text)
    volume = volume_match.group(1) if volume_match else ""

 # Extract the page
    page_match = re.search(r",\s(\d+G)$", publication_text)
    page = page_match.group(1) if page_match else ""

    publication_result = dict()
    publication_result["Publication Title"] = publication_title
    publication_result["Authors"] = authors
    publication_result["Year"] = year
    publication_result["Journal"] = journal
    publication_result["Volume"] = volume
    publication_result["Page"] = page



    return publication_result




In [93]:


def check_publication_dict(sent_dict):
    if sent_dict['Publication Title']=='' and len(sent_dict['Authors'])==0 and sent_dict['Year']=='' and  sent_dict['Journal']=='' and  sent_dict['Volume']=='' and  sent_dict['Page']=='' :
        return False
    return True





In [94]:


def handled_publications(sent_publications_list):
    all_publications_list = []

    for item in sent_publications_list:
        temp_publications_list =[]


            # Split text into publications
        publications = re.split(r"\x03[0-9]", item)

        for text in publications:
                #print("text :::",text)
            temp_dict = get_publication_dict(text)
            if check_publication_dict(temp_dict)==True:
                temp_publications_list.append(temp_dict)
        if len(temp_publications_list)!=0:
            all_publications_list.append(temp_publications_list)
    if len(all_publications_list)==0:
        all_publications_list = handle_all_list_of_publications(sent_publications_list)
    return all_publications_list



In [ ]:


gotten_publications_list = handled_publications(new_dict_for_division['publications'])


gotten_publications_list





In [ ]:
new_dict_for_division['co-authored papers']

### do it later




In [95]:


def handle_confereance_text(sent_text):

 # Split the text into separate conference entries
    conference_entries = re.split(r'\x03\d', sent_text)[1:]

    conference_data = []

    for entry in conference_entries:
        segments = entry.split(',')

    # Extract conference title
        conference_title = segments[0].strip() if len(segments) > 0 else None

    # Extract year
        year_match = re.search(r'\d{4}', entry)
        year = int(year_match.group()) if year_match else None

    # Extract presentation information
        presentation = ','.join(segments[1:-1]).strip() if len(segments) > 2 else None

    # Extract location
        location = segments[-1].strip() if len(segments) > 1 else None

        conference_data.append({
            'conference_title': conference_title,
            'presentation': presentation,
            'year': year,
            'location': location,
        })

    return conference_data




In [96]:

def handle_confereance(sent_conference_list):
    result_cconferences = []
    for item in sent_conference_list:
        res = handle_confereance_text(item)
        if len(res)>0:
            result_cconferences.append(res)
    return result_cconferences





In [ ]:
conferences_all = handle_confereance(new_dict_for_division['conference abstracts and proceedings'])

conferences_all





In [133]:
import re
import spacy

# Load the spaCy English NER model
nlp = spacy.load("en_core_web_sm")

def find_dates_and_organizations(input_text):
    # Define regular expression patterns to match various date formats
    # date_patterns = [
    #     r'\b(?:\d{1,2}[-/]\d{1,2}[-/]\d{2,4}|\d{1,2}/\d{1,2}\s*-\s*\d{1,2}/\d{2,4}|\w+\s+\d{1,2}(?:-\s*\d{1,2})?,\s+\d{4})\b',
    #     r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\.\s+\d{1,2}(?:-\s*\d{1,2})?,\s+\d{4}\b',
    #     r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{1,2}-\d{1,2}(?:\s*-\s*(?:\d{1,2}/)?\d{4})?,?\s*(?:-\s*present)?\b',  # Match dates with "present"
    #     r'\b\d{1,2}/\d{2,4}\s*-\s*present\b',  # Match dates like "2/09 - present"
    #     r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{1,2}\s*-\s*\d{1,2},\s*\d{4}\b',  # Match dates like "Jan 2 - 23, 2023"
    #     r'\b\w+\s+\d{1,2}-\w+\s+\d{1,2},\s*\d{4}\b'  # Match dates like "May 28-June 3, 2023"
    # ]
    date_patterns = [
    r'\b(?:\d{1,2}[-/]\d{1,2}[-/]\d{2,4}|\d{1,2}/\d{1,2}\s*-\s*\d{1,2}/\d{2,4}|\w+\s+\d{1,2}(?:-\s*\d{1,2})?,\s+\d{4})\b',
    r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\.\s+\d{1,2}(?:-\s*\d{1,2})?,\s+\d{4}\b',
    r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{1,2}-\d{1,2}(?:\s*-\s*(?:\d{1,2}/)?\d{4})?,?\s*(?:-\s*present)?\b',  # Match dates with "present"
    r'\b\d{1,2}/\d{2,4}\s*-\s*present\b',  # Match dates like "2/09 - present"
    r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{1,2}\s*-\s*\d{1,2},\s*\d{4}\b',  # Match dates like "Jan 2 - 23, 2023"
    r'\b\w+\s+\d{1,2}-\w+\s+\d{1,2},\s*\d{4}\b',  # Match dates like "May 28-June 3, 2023"
    r'\b\(\d{4}\)\b',  # Match dates like "(2017)"
    r'\d{1,2}/\d{4}-\d{1,2}/\d{4}\b',  # Match dates like "12/2016-12/2017"
    r'\d{4}-\d{4}\b'  # Match dates like "2014-2016"
    ]


    # Use a list to store unique dates and their corresponding topics and organizations
    unique_data = []

    # Split the input text into words
    words = input_text.split()

    # Initialize a string to accumulate words
    current_string = ""

    # Initialize variables to keep track of the current date and previous date
    current_date = None
    previous_date = None

    # Iterate through the words in the input text
    for word in words:
        current_string += " " + word  # Add the current word to the string with a space

        # Check if the current string contains a date using the date patterns
        for date_pattern in date_patterns:
            dates = re.findall(date_pattern, current_string)
            if dates:
                current_date = dates[-1]  # Take the most recent date
                if current_date != previous_date:
                    topic = current_string.replace(current_date, "").strip()  # Assign the string to the topic
                    if topic:
                        # Use spaCy to extract organizations from the topic
                        doc = nlp(topic)
                        organizations = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
                        # Remove the extracted organizations from the topic
                        for org in organizations:
                            topic = topic.replace(org, "").strip()

                        temp = {'date': current_date, 'topic': topic, 'organizations': organizations}
                        # print(temp)
                        unique_data.append(temp)
                    current_string = ""  # Start with a new empty string after finding a new date
                break

        # Update the previous date
        previous_date = current_date

    return unique_data





In [134]:


def handle_conference_abstracts_and_proceedings(sent_list):
    results = []
    for item in sent_list:
        res = find_dates_and_organizations(item)
        if len(res)>0:
            results.append(res)
    return results








In [136]:

all_handle_conference_abstracts_and_proceedings = handle_conference_abstracts_and_proceedings(new_dict_for_division['conference abstracts and proceedings'])


all_handle_conference_abstracts_and_proceedings




[[{'date': '07/2013-07/2014',
   'topic': "Conference of the  in .Reynolds KA, Canales RA,  (2017). Modeling Viral Pathogen Exposure and Risk Reductions for  Interventions, Lewis Stokes Midwest Center of Excellence (LSMCE) Conference. Wagoner R, Lopez-Galvez N, , DeZapien J, Rosales C, Canales RA, Ortega I,  (2017). Heat and Hydration Assessment of Migrant Grape-Workers in Sonora, Mexico, . O'Rourke MK , Hadeed S , Honanie G , Mahkewa A ,  ,  , Alshammari MA , Canales RA , Harri R (2017).  in Households on the Hopi 2.5 Reservation, . M, Reynolds KA, Canales RA (2018). Predicting s and Optimizing Hygiene Protocols Using a Modeling Approach,  in  and Epidemiology Annual Conference. M, Reynolds KA, Canales RA (2018). Using Simulation Modeling to Estimate the Effect of Human Behaviors and Cleaning Interventions on Infection Risk, American Society of Safety Engineers. M, Reynolds KA, Gerba CP, Canales RA (2018). How Much Are Surfaces to Blame in ?, . M, Reynolds KA, Canales RA (2018). Valid

In [99]:


def handle_all_sections(new_dict_for_division):
    all_extracted_data = {
    'contact information':[], #1
    'research areas':[], #2
    'education':[], #3
    'appointments':[], #4
    'proposals/grants':[], #5
    'teaching/ outreach':[], #6
    'students':[], #7
    'honors/awards':[], #8
    'selected talks/meetings':[], #9
    'services':[], #10
    'observing experience':[], #11
    'publications':[], #12
    'co-authored papers':[], #13 not implemented
    'conference abstracts and proceedings':[], #14
    }



    for item_list in new_dict_for_division.keys():
        data = []
        if item_list=='contact information':
            data = handled_contact_information(new_dict_for_division['contact information'])
            if len(data)>0:
                all_extracted_data[item_list] = data
        elif item_list=='education':
            data = handled_all_educations(new_dict_for_division['education'])
            if len(data)>0:
                all_extracted_data[item_list] = data
        elif item_list=='research areas':
            data = handle_all_research_area(new_dict_for_division['research areas'])
            if len(data)>0:
                all_extracted_data[item_list] = data
        elif item_list=='appointments':
            data = handle_all_appoinments(new_dict_for_division['appointments'])
            if len(data)>0:
                all_extracted_data[item_list] = data
        elif item_list=='proposals/grants':
            data = handle_all_grants(new_dict_for_division['proposals/grants'])
            if len(data)>0:
                all_extracted_data[item_list] = data
            else:
                data = handle_all_grants_02(new_dict_for_division['proposals/grants'])
                all_extracted_data[item_list] = data
        elif item_list=='teaching/ outreach':
            data = all_handled_teaching_outreach(new_dict_for_division['teaching/ outreach'])
            if len(data)>0:
                all_extracted_data[item_list] = data
        elif item_list=='students':
            data = extract_student_data(new_dict_for_division['students'])
            if len(data)>0:
                all_extracted_data[item_list] = data
        elif item_list=='honors/awards':
            data = handled_honors_awards(new_dict_for_division['honors/awards'])
            if len(data)>0:
                all_extracted_data[item_list] = data
        elif item_list=='selected talks/meetings':
            data = handled_talks_meetings(new_dict_for_division['selected talks/meetings'])
            if len(data)>0:
                all_extracted_data[item_list] = data
        elif item_list=='services':
            data = handled_services_02(new_dict_for_division['services'])
            if len(data)>0:
                all_extracted_data[item_list] = data
        elif item_list=='observing experience':
            data = handled_observing_experience(new_dict_for_division['observing experience'])
            if len(data)>0:
                all_extracted_data[item_list] = data
        elif item_list=='publications':
            data = handled_publications(new_dict_for_division['publications'])
            if len(data)>0:
                all_extracted_data[item_list] = data
        elif item_list=='co-authored papers':
            # data = handled_publications(new_dict_for_division['publications'])
            # if len(data)>0:
            #     all_extracted_data[item_list] = data
            print("Please implement the function. Not implemented.")
        elif item_list=='conference abstracts and proceedings':
            data = handle_conference_abstracts_and_proceedings(new_dict_for_division['conference abstracts and proceedings'])
            if len(data)>0:
                all_extracted_data[item_list] = data
            else:
                data = handle_confereance(new_dict_for_division['conference abstracts and proceedings'])
                all_extracted_data[item_list] = data
        else:
            print("New section. Please implement method.")

    return all_extracted_data







In [142]:
new_dict_for_division.keys()

dict_keys(['contact information', 'honors/awards', 'publications', 'conference abstracts and proceedings'])

In [114]:
new_dict_for_division['education']

KeyError: ignored

In [150]:

all_extracted_data = handle_all_sections(new_dict_for_division)

all_extracted_data



UnboundLocalError: ignored

In [138]:
#pickle_path = '/content/drive/MyDrive/CV_extract_data/pickle_data/one_CV_data_by_section_dict.pickle'



#pdf_path = '/content/drive/MyDrive/private dataset(no sharing)/agaspar_cv.pdf'
file_name = os.path.basename(pdf_path)
file_name_without_extension = os.path.splitext(file_name)[0]
print(file_name_without_extension)


pickle_path = '/content/drive/MyDrive/CV_extract_data/pickle_data/' + file_name_without_extension + "_data" + '.pickle'





Canales_Robert_CV


In [139]:




with open(pickle_path, 'wb') as file:
    pickle.dump(all_extracted_data, file)


In [140]:
with open(pickle_path, 'rb') as file:
    loaded_data = pickle.load(file)

print(loaded_data)


{'contact information': {'Names': ['Honors', 'Civil Engineering', 'Civil', 'Statistics', 'Civil', 'Framework Advisor', 'James O.', 'Sustainable Design', 'Boston Architectural College', 'Risk Program', 'Harvard School', 'Science Program', 'Civil', 'Science Program', 'Civil', 'Parsons School', 'Eugene Lang College', 'Policy Division', 'Mel', 'Enid Zuckerman College'], 'email': [], 'phone_number': []}, 'research areas': [], 'education': [], 'appointments': [], 'proposals/grants': [], 'teaching/ outreach': [], 'students': [], 'honors/awards': [[{'Year:': '1992', 'Honors/Awards:': 'Texas Achievement Honors Award (4 years)'}, {'Year:': '1996', 'Honors/Awards:': 'Tau Beta Pi Engineering Honor Society'}, {'Year:': '1996', 'Honors/Awards:': 'National Science Foundation (NSF) Graduate Fellowship'}, {'Year:': '1998', 'Honors/Awards:': 'Karl E. Schwarz Graduate Award, Stanford University'}, {'Year:': '2001', 'Honors/Awards:': 'Centennial Teaching Award, Stanford University'}, {'Year:': '2010', 'Ho

In [141]:

i=1
for item in loaded_data.keys():
    print(item, " # ", i ," :: ", loaded_data[item])
    i+=1




contact information  #  1  ::  {'Names': ['Honors', 'Civil Engineering', 'Civil', 'Statistics', 'Civil', 'Framework Advisor', 'James O.', 'Sustainable Design', 'Boston Architectural College', 'Risk Program', 'Harvard School', 'Science Program', 'Civil', 'Science Program', 'Civil', 'Parsons School', 'Eugene Lang College', 'Policy Division', 'Mel', 'Enid Zuckerman College'], 'email': [], 'phone_number': []}
research areas  #  2  ::  []
education  #  3  ::  []
appointments  #  4  ::  []
proposals/grants  #  5  ::  []
teaching/ outreach  #  6  ::  []
students  #  7  ::  []
honors/awards  #  8  ::  [[{'Year:': '1992', 'Honors/Awards:': 'Texas Achievement Honors Award (4 years)'}, {'Year:': '1996', 'Honors/Awards:': 'Tau Beta Pi Engineering Honor Society'}, {'Year:': '1996', 'Honors/Awards:': 'National Science Foundation (NSF) Graduate Fellowship'}, {'Year:': '1998', 'Honors/Awards:': 'Karl E. Schwarz Graduate Award, Stanford University'}, {'Year:': '2001', 'Honors/Awards:': 'Centennial Teac